In [1]:
import numpy
import scipy.signal
import scipy.optimize
import matplotlib.pyplot as plt
%matplotlib inline

We start with a linear model of the system

In [2]:
G11 = scipy.signal.lti([-10], [40, 1])
G12 = scipy.signal.lti([5], [5,1])


In [3]:
M = 18  # Control horizon
P = 72  # Prediction horizon

In [4]:
DeltaT = 1  # Sampling rate

In [5]:
tcontinuous = numpy.linspace(0, DeltaT*P, 1000)  # some closely spaced time points
tpredict = numpy.arange(0, P*DeltaT, DeltaT)   # discrete points at prediction horizon


In [6]:
u0 = 20 # initial value of MV
y0 = 100 # initial value of CV.
u = u0*numpy.ones(M)  # u is an array of length  M ( control horizon )
r = y0*numpy.ones(P)


In [8]:
Q= 1 # CV Weight 
R= 10 # MV Weight 

In [9]:
def extend(u):
    return numpy.concatenate([u, numpy.repeat(u[-1], P-M)])

In [12]:
def prediction(u, t=tpredict):
    t, y, x = scipy.signal.lsim(G, u, t, X0=0, interp=False)
    return y

In [13]:
def objective(u):
    y = prediction(extend(u))
    return (sum((r - y)**2)+ sum ((numpy.diff(u)**2)*(R**2)))

Now we figure out a set of moves which will minimise our objective function

In [14]:
uopt = scipy.optimize.minimize(objective, u).x


NameError: name 'G' is not defined

Resample the discrete output to continuous time

In [ ]:
ucont = extend(uopt)[((tcontinuous-0.01)//DeltaT).astype(int)]


Plot the move plan and the output. Notice that we are getting exactly the output we want at the sampling times.

In [ ]:
plt.figure()
plt.plot(tcontinuous, ucont)
plt.xlabel("MV")
plt.xlim([0, DeltaT*(P+1)])
plt.figure()
plt.plot(tcontinuous, prediction(ucont, tcontinuous), 
         tpredict, prediction(extend(uopt)),
         tpredict, r,
         )
plt.xlabel("CV")

In [ ]:
plt.plot(tcontinuous, prediction(ucont, tcontinuous), 
         tpredict, prediction(extend(uopt)),
         tpredict, r,
         )

### things to do
1. develop a steadystate solver that calculates the steady state targets
inputs:
    . model
    . initial conditions of MV 
    . initial conditions of CVs
    . CV targets. 
    . CV priorities. 
    . CV Weights. 
  ouputs
  . CV targets
  . MV Targets
 2. devleop constrained optimizer.  ( max move constraints, vavle position constraints)
 3. estimate state using kalman 
 
 
 Flow chart 
 1. calculate the set of optimal moves 
 2. implement the first move
 3. implement the state estimator 
 4. update the predictions, calculate the new set of moves
 5.